In [127]:
import numpy as np
import pandas as pd
import astropy.io.fits as fits
import healpy as hp
from scipy import integrate
import matplotlib.pyplot as plt
from astropy.cosmology import Planck15 as cosmo
from scipy.interpolate import interp1d
from scipy.optimize import curve_fit
#from completeness import create_completeness_dataframe

In [128]:
# Loading the data
galaxy_catalog_raw = fits.open('/home/farnoosh/Nextcloud/Master_Thesis_all/codes/data/GAMA/merged/StellarMass-gkvScience/mergedStellarMass-gkvScience')[1].data    # galaxies
efeds_raw = fits.open('/home/farnoosh/Nextcloud/Master_Thesis_all/codes/data/eFEDS/Mathias_Klug/efeds_members.fit')[1].data    # clusters

In [129]:
# Masks
cluster_mask =(
            ~np.isnan(efeds_raw['VDISP'])   # mask clusters which doesn't have VDISP values
 )


galaxy_mask = (
         (galaxy_catalog_raw['uberclass'] == 1 ) & #galaxy
         (galaxy_catalog_raw['duplicate'] == False) &
         (galaxy_catalog_raw['mask'] == False) &
         (galaxy_catalog_raw['NQ'] > 2) &
         (galaxy_catalog_raw['SC'] > 7) &
         (galaxy_catalog_raw['mstar'] > 0) &
         (galaxy_catalog_raw['RAcen'] > 126.5) &
         (galaxy_catalog_raw['RAcen'] < 145.1) &
         (galaxy_catalog_raw['Deccen'] > -2.6) &
         (galaxy_catalog_raw['Deccen'] < 5.8) &
         (galaxy_catalog_raw['Z'] < 1.31) &
         (galaxy_catalog_raw['starmask'] == False)
)

galaxy_catalog = galaxy_catalog_raw[galaxy_mask]
efeds_catalog = efeds_raw[cluster_mask]
print('number of the galaxies after masking: ',len(galaxy_catalog))
print('number of the clusters after masking: ', len(efeds_catalog))

number of the galaxies after masking:  36801
number of the clusters after masking:  282


In [130]:
# Constants
H0 = 70
c = 3e5
arcmin_to_rad = np.pi / 180 / 60

In [135]:
# Functions
def distance_from_redshift(z):
    return c * z / H0

def cluster_volume(cluster_radius_Mpc):
    return (4/3) * np.pi * cluster_radius_Mpc**3

def redshift_comparison(cluster_z, cluster_Velocity_Dispersion, galaxy_z, threshold_factor=1):
    delta_z = (cluster_Velocity_Dispersion / c) * threshold_factor
    return np.abs(galaxy_z - cluster_z) <= delta_z

def add_z_passed_galaxies_column(cluster_df, galaxy_df, threshold_factor=1):
    """
    Adds a column to cluster_df containing the IDs of galaxies that pass the redshift comparison criteria for each cluster,
    and another column showing the number of galaxies that passed the criteria for each cluster.

    Parameters:
    - cluster_df (pd.DataFrame): DataFrame containing cluster information.
    - galaxy_df (pd.DataFrame): DataFrame containing galaxy information.
    - threshold_factor (float): Factor to adjust the redshift comparison criteria.

    Returns:
    - cluster_df (pd.DataFrame): Updated DataFrame with the 'z_passed_galaxies' and 'num_passed_galaxies' columns.
    """

    def redshift_comparison(cluster_z, cluster_Velocity_Dispersion, galaxy_z):
        delta_z = (cluster_Velocity_Dispersion / c) * threshold_factor
        return np.abs(galaxy_z - cluster_z) <= delta_z

    # Create an empty list to store the IDs of galaxies that pass the criteria for each cluster
    z_passed_galaxies_list = []
    num_passed_galaxies_list = []

    # Iterate over each cluster
    for i, cluster in cluster_df.iterrows():
        # Get the delta_z for the current cluster
        delta_z_cluster = (cluster['cluster_Velocity_Dispersion'] / c) * threshold_factor

        # Create an empty list to store the IDs of galaxies that pass the criteria for the current cluster
        passed_galaxies_cluster = []

        # Iterate over each galaxy
        for j, galaxy in galaxy_df.iterrows():
            # Check if the galaxy passes the redshift comparison criteria for the current cluster
            if redshift_comparison(cluster['z'], cluster['cluster_Velocity_Dispersion'], galaxy['z']):
                # Add the ID of the galaxy to the list
                passed_galaxies_cluster.append(galaxy['g_ID'])

        # Add the list of IDs to the z_passed_galaxies_list
        z_passed_galaxies_list.append(passed_galaxies_cluster)
        num_passed_galaxies_list.append(len(passed_galaxies_cluster))

    # Add new columns to cluster_df with the list of IDs of galaxies that passed the criteria
    cluster_df['z_passed_galaxies'] = z_passed_galaxies_list
    cluster_df['num_passed_galaxies'] = num_passed_galaxies_list

    return cluster_df


def angular_separation(ra1, dec1, ra2, dec2):
    ra1, dec1, ra2, dec2 = map(np.radians, [ra1, dec1, ra2, dec2])
    delta_RA = ra2 - ra1
    delta_DEC = dec2 - dec1
    w = np.sin(delta_DEC/2.0)**2 + np.cos(dec1) * np.cos(dec2) * np.sin(delta_RA/2.0)**2
    return 2 * np.arcsin(np.sqrt(w)) # in degrees

def projected_distance(cluster_ra, cluster_dec, cluster_z, galaxy_ra, galaxy_dec):
    angular_dist = angular_separation(cluster_ra, cluster_dec, galaxy_ra, galaxy_dec)   # in degrees
    angular_distance_radian = np.radians(angular_dist)  # in Radian
    D_A = cosmo.angular_diameter_distance(cluster_z).value
    projected_distance_Mpc = D_A * angular_distance_radian
    return projected_distance_Mpc


# def is_galaxy_in_cluster(galaxy_ra, galaxy_dec, galaxy_z, cluster_ra, cluster_dec, cluster_z, cluster_radius_Mpc):
#     projected_dist_Mpc = projected_distance(cluster_ra, cluster_dec, cluster_z, galaxy_ra, galaxy_dec)
#     if projected_dist_Mpc <= cluster_radius_Mpc:
#         if redshift_comparison(cluster_z, 0, galaxy_z):
#             return True
#     return False
#
# def apply_redshift_mask(cluster_df, galaxy_df):
#     galaxy_within_cluster_mask = np.zeros(len(galaxy_df), dtype=bool)
#     for i, galaxy in galaxy_df.iterrows():
#         for j, cluster in cluster_df.iterrows():
#             if is_galaxy_in_cluster(galaxy['RA'], galaxy['DEC'], galaxy['z'],
#                                     cluster['RA'], cluster['DEC'], cluster['z'], cluster['cluster_radius_Mpc']):
#                 galaxy_within_cluster_mask[i] = True
#                 break  # Once a galaxy is found within a cluster, move to the next galaxy
#     return galaxy_df[galaxy_within_cluster_mask]

In [136]:
# Dataframe for clusters
cluster_df = pd.DataFrame()
cluster_df['c_ID'] = efeds_catalog['ID-SRC'].byteswap().newbyteorder()
cluster_df['RA'] = (efeds_catalog['RA']).byteswap().newbyteorder()    # degrees
cluster_df['DEC'] = (efeds_catalog['DEC']).byteswap().newbyteorder()   # degrees
cluster_df['z'] = efeds_catalog['ZBEST'].byteswap().newbyteorder()
cluster_df['distance_Mpc'] = distance_from_redshift(cluster_df['z'])
cluster_df['cluster_radius_Mpc'] = efeds_catalog['R_LAMBDA']
cluster_df['cluster_volume_Mpc3'] = cluster_volume(cluster_df['cluster_radius_Mpc'])
cluster_df['cluster_Velocity_Dispersion'] = efeds_catalog['VDISP']  # (Km/s)

In [137]:
# Dataframe for galaxies
galaxy_df = pd.DataFrame()
galaxy_df['g_ID'] = galaxy_catalog['uberID'].byteswap().newbyteorder()
galaxy_df['RA'] = galaxy_catalog['RAcen'].byteswap().newbyteorder()  #degrees
galaxy_df['DEC'] = galaxy_catalog['Deccen'].byteswap().newbyteorder()    #degrees
galaxy_df['z'] = galaxy_catalog['Z'].byteswap().newbyteorder()

In [138]:
# Call the function to add the 'z_passed_galaxies' and 'num_passed_galaxies' columns to cluster_df
cluster_df_updated = add_z_passed_galaxies_column(cluster_df, galaxy_df)

# Display the updated cluster DataFrame
print(cluster_df_updated)

      c_ID         RA      DEC       z  distance_Mpc  cluster_radius_Mpc  \
0     4800  127.03664 -0.16772  0.0762    326.571429            0.664964   
1     7991  127.16920 -0.08355  0.3197   1370.142857            0.598947   
2     5601  127.79401  1.93784  0.4195   1797.857143            0.924255   
3     2659  127.85804  1.92590  0.6845   2933.571429            0.992066   
4     3725  127.90807  0.77581  0.3023   1295.571429            0.698281   
..     ...        ...      ...     ...           ...                 ...   
277   1761  143.94326 -0.02110  0.3497   1498.714286            0.810680   
278  11748  144.28102  3.80886  0.2709   1161.000000            0.788915   
279     38  144.30357  3.28104  0.2474   1060.285714            0.681306   
280  10322  144.62717  4.25640  0.2622   1123.714286            0.672590   
281   1594  144.90960  4.37172  0.3687   1580.142857            0.783890   

     cluster_volume_Mpc3  cluster_Velocity_Dispersion  \
0               1.231640      